In [1]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}


In [2]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [3]:
def load_data(train_size=0.8,test_size=0.2):
    X, y = [], []
    try :
        for file in glob.glob("C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset/Actor_*/*.wav"):
            # get the base name of the audio file
            print(file)
            basename = os.path.basename(file)
            print(basename)
          # get the emotion label
            emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
            if emotion not in AVAILABLE_EMOTIONS:
                continue
          # extract speech features
            features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
            X.append(features)
            l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
            y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size,train_size=train_size,random_state=7)

In [4]:
X_train, X_test, y_train, y_test = load_data()

print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])

C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-01-01-01.wav
03-01-01-01-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-01-02-01.wav
03-01-01-01-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-02-01-01.wav
03-01-01-01-02-01-01.wav


C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[ 3.0517578e-05  3.0517578e-05  3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-02-02-01.wav
03-01-01-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-01-01-01.wav
03-01-02-01-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-01-02-01.wav
03-01-02-01-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-02-01-01.wav
03-01-02-01-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-02-02-01.wav
03-01-02-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-01-01-01.wav
03-01-02-02-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-01-02-01.wav
03-01-02-02-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-02-01-01.wav
03-01-02-02-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-

C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-01-02-01-01.wav
03-01-03-01-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-01-02-02-01.wav
03-01-03-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-02-01-01-01.wav
03-01-03-02-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-02-01-02-01.wav
03-01-03-02-01-02-01.wav


...

C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-01-02-01-23.wav
03-01-03-01-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-01-02-02-23.wav
03-01-03-01-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-02-01-01-23.wav
03-01-03-02-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-02-01-02-23.wav
03-01-03-02-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-02-02-01-23.wav
03-01-03-02-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-03-02-02-02-23.wav
03-01-03-02-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-04-01-01-01-23.wav
03-01-04-01-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-04-01-01-02-23.wav
03-01-04-01-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-

C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05  0.0000000e+00 ...  3.0517578e-05
  3.0517578e-05  3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-01-01-02-01-24.wav
03-01-01-01-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-01-01-02-02-24.wav
03-01-01-01-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-01-01-01-24.wav
03-01-02-01-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-01-01-02-24.wav
03-01-02-01-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-01-02-01-24.wav
03-01-02-01-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-01-02-02-24.wav
03-01-02-01-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-02-01-01-24.wav
03-01-02-02-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-02-02-01-02-24.wav
03-01-02-02-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-

C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.5258789e-04
 -1.5258789e-04 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-01-01-02-24.wav
03-01-05-01-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-01-02-01-24.wav
03-01-05-01-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-01-02-02-24.wav
03-01-05-01-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-01-01-24.wav
03-01-05-02-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-01-02-24.wav
03-01-05-02-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-02-01-24.wav
03-01-05-02-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-02-02-24.wav
03-01-05-02-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-06-01-01-01-24.wav
03-01-06-01-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-

In [5]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [6]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((537, 180), (537,), (135, 180), (135,))

In [7]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [8]:
x_traincnn.shape,x_testcnn.shape

((537, 180, 1), (135, 180, 1))

In [9]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

from tensorflow import keras
from keras import optimizers
%load_ext tensorboard
%matplotlib inline




model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
optimizer=keras.optimizers.RMSprop(learning_rate=0.001,
    rho=0.9,
    epsilon=None,
    centered=False,
    name='RMSprop'
)

#opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 180, 128)          768       
                                                                 
 activation (Activation)     (None, 180, 128)          0         
                                                                 
 dropout (Dropout)           (None, 180, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 22, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_1 (Activation)   (None, 22, 128)           0         
                                                        

In [11]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [31]:
#!rm -rf ./logs/
import tensorflow
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8960), started 1 day, 2:56:39 ago. (Use '!kill 8960' to kill it.)

In [13]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/50
27/27 [==============================] - 3s 46ms/step - loss: 7.6534 - accuracy: 0.3073 - val_loss: 3.7073 - val_accuracy: 0.2889
Epoch 2/50
27/27 [==============================] - 1s 25ms/step - loss: 2.9742 - accuracy: 0.3594 - val_loss: 3.1445 - val_accuracy: 0.3481
Epoch 3/50
27/27 [==============================] - 1s 25ms/step - loss: 1.9623 - accuracy: 0.4134 - val_loss: 1.1853 - val_accuracy: 0.3630
Epoch 4/50
27/27 [==============================] - 1s 25ms/step - loss: 1.4010 - accuracy: 0.4451 - val_loss: 1.5197 - val_accuracy: 0.3407
Epoch 5/50
27/27 [==============================] - 1s 24ms/step - loss: 1.3161 - accuracy: 0.4600 - val_loss: 1.0305 - val_accuracy: 0.5481
Epoch 6/50
27/27 [==============================] - 1s 24ms/step - loss: 1.0860 - accuracy: 0.5102 - val_loss: 1.1469 - val_accuracy: 0.5704
Epoch 7/50
27/27 [==============================] - 1s 25ms/step - loss: 1.1023 - accuracy: 0.4898 - val_loss: 1.0632 - val_accuracy: 0.4963
Epoch 8/50
27

In [14]:
em=['happy','sad','neutral','angry']

In [15]:
predictions = model.predict(x_testcnn)
n=predictions[1]
print(em[1])
print(n)

5/5 [==============================] - 0s 6ms/step
sad
[1.3474981e-01 1.6509653e-12 1.5825342e-29 1.1531325e-10 8.6525017e-01
 1.3481200e-29 5.8116304e-31 2.4360877e-29]


In [16]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 7ms/step - loss: 0.8971 - accuracy: 0.6889
Restored model, accuracy: 68.89%


In [17]:
filename = "C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset/Actor_02/03-01-01-01-02-01-02.wav"
    # record the file (start talking)
    #record_to_file(filename)
    # extract features and reshape it
features = np.array(extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
f=np.expand_dims(features,axis=2)
result = model.predict(f)[0]
    # show the result !
print("result :",em[int(result[0])])

1/1 [==============================] - 0s 31ms/step
result : happy


C:\Users\hp\AppData\Local\Temp\ipykernel_6296\1163463125.py:31: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


**Second**

In [18]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from tensorflow import keras
from keras import optimizers


um = Sequential()

um.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
um.add(Activation('relu'))
um.add(Dropout(0.25))
um.add(MaxPooling1D(pool_size=(8)))

um.add(Conv1D(128, 5,padding='same',))                  #2
um.add(Activation('relu'))
um.add(MaxPooling1D(pool_size=(8)))
um.add(Dropout(0.25))

um.add(Conv1D(128, 5,padding='same',))                  #3
um.add(Activation('relu'))
um.add(Dropout(0.25))

um.add(Flatten())
um.add(Dense(8))                                        #4                      
um.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(learning_rate=0.00005,epsilon=None,rho=0.9,decay=0.0)



In [19]:
um.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_3 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_2 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_4 (Activation)   (None, 22, 128)           0         
                                                      

In [20]:
um.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [21]:
umhistory=um.fit(x_traincnn, y_train, batch_size=20, epochs=100, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/100
27/27 [==============================] - 3s 33ms/step - loss: 10.0141 - accuracy: 0.1899 - val_loss: 1.4867 - val_accuracy: 0.3333
Epoch 2/100
27/27 [==============================] - 1s 20ms/step - loss: 7.3589 - accuracy: 0.2439 - val_loss: 1.3917 - val_accuracy: 0.4148
Epoch 3/100
27/27 [==============================] - 1s 23ms/step - loss: 6.9053 - accuracy: 0.2142 - val_loss: 1.4032 - val_accuracy: 0.4444
Epoch 4/100
27/27 [==============================] - 1s 25ms/step - loss: 5.8683 - accuracy: 0.2533 - val_loss: 1.9117 - val_accuracy: 0.4444
Epoch 5/100
27/27 [==============================] - 1s 25ms/step - loss: 5.3614 - accuracy: 0.2961 - val_loss: 1.4411 - val_accuracy: 0.5037
Epoch 6/100
27/27 [==============================] - 1s 25ms/step - loss: 4.3319 - accuracy: 0.3091 - val_loss: 1.2958 - val_accuracy: 0.4519
Epoch 7/100
27/27 [==============================] - 1s 25ms/step - loss: 4.6236 - accuracy: 0.2663 - val_loss: 1.3137 - val_accuracy: 0.4593
Epoch

In [22]:
#!rm -rf ./logs/
import tensorflow
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8960), started 1 day, 2:55:54 ago. (Use '!kill 8960' to kill it.)

In [23]:
loss, acc = um.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 9ms/step - loss: 1.1014 - accuracy: 0.5259
Restored model, accuracy: 52.59%


**Fourth**

In [24]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from keras import optimizers


sm = Sequential()

sm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
sm.add(Activation('relu'))
sm.add(Dropout(0.1))
sm.add(MaxPooling1D(pool_size=(8)))

sm.add(Conv1D(128, 5,padding='same',))                  #2
sm.add(Activation('relu'))
sm.add(MaxPooling1D(pool_size=(8)))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #3
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #4
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Flatten())
sm.add(Dense(8))                                        #5                     
sm.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00005,epsilon=None,rho=0.9,decay=0.0)


C:\Users\hp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [25]:
sm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_7 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_5 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_8 (Activation)   (None, 22, 128)           0         
                                                      

In [26]:
sm.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [27]:
smhistory=sm.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/50
27/27 [==============================] - 4s 43ms/step - loss: 3.0166 - accuracy: 0.2104 - val_loss: 1.4519 - val_accuracy: 0.3407
Epoch 2/50
27/27 [==============================] - 1s 27ms/step - loss: 2.1454 - accuracy: 0.2663 - val_loss: 1.3476 - val_accuracy: 0.4296
Epoch 3/50
27/27 [==============================] - 1s 28ms/step - loss: 2.0680 - accuracy: 0.2719 - val_loss: 1.3594 - val_accuracy: 0.4222
Epoch 4/50
27/27 [==============================] - 1s 27ms/step - loss: 1.8225 - accuracy: 0.3035 - val_loss: 1.3261 - val_accuracy: 0.4222
Epoch 5/50
27/27 [==============================] - 1s 27ms/step - loss: 1.6746 - accuracy: 0.3073 - val_loss: 1.2878 - val_accuracy: 0.4519
Epoch 6/50
27/27 [==============================] - 1s 28ms/step - loss: 1.7217 - accuracy: 0.2905 - val_loss: 1.2452 - val_accuracy: 0.4593
Epoch 7/50
27/27 [==============================] - 1s 28ms/step - loss: 1.5960 - accuracy: 0.3259 - val_loss: 1.2544 - val_accuracy: 0.4519
Epoch 8/50
27

In [28]:
#!rm -rf ./logs/
import tensorflow
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8960), started 1 day, 2:56:39 ago. (Use '!kill 8960' to kill it.)

In [29]:
loss, acc = sm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 9ms/step - loss: 1.0364 - accuracy: 0.5630
Restored model, accuracy: 56.30%


In [30]:
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import callbacks

In [39]:
model_LSTM=Sequential()
model_LSTM.add(layers.LSTM(64,return_sequences=True,input_shape=(180,1)))
model_LSTM.add(layers.LSTM(64))
model_LSTM.add(layers.Dense(8,activation='softmax'))
print(model_LSTM.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 180, 64)           16896     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 8)                 520       
                                                                 
Total params: 50,440
Trainable params: 50,440
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
model_LSTM.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [41]:
umhistory=model_LSTM.fit(x_traincnn, y_train, batch_size=20, epochs=100, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/100
27/27 [==============================] - 15s 246ms/step - loss: 2.0730 - accuracy: 0.2868 - val_loss: 2.0664 - val_accuracy: 0.3111
Epoch 2/100
27/27 [==============================] - 5s 177ms/step - loss: 2.0592 - accuracy: 0.2831 - val_loss: 2.0492 - val_accuracy: 0.3037
Epoch 3/100
27/27 [==============================] - 5s 181ms/step - loss: 2.0371 - accuracy: 0.2812 - val_loss: 2.0200 - val_accuracy: 0.3037
Epoch 4/100
27/27 [==============================] - 5s 179ms/step - loss: 1.9975 - accuracy: 0.2812 - val_loss: 1.9638 - val_accuracy: 0.3037
Epoch 5/100
27/27 [==============================] - 5s 185ms/step - loss: 1.9119 - accuracy: 0.2812 - val_loss: 1.8328 - val_accuracy: 0.3037
Epoch 6/100
27/27 [==============================] - 5s 182ms/step - loss: 1.7316 - accuracy: 0.2812 - val_loss: 1.6157 - val_accuracy: 0.3037
Epoch 7/100
27/27 [==============================] - 5s 187ms/step - loss: 1.5800 - accuracy: 0.2812 - val_loss: 1.5073 - val_accuracy: 0.303

Epoch 58/100
27/27 [==============================] - 4s 164ms/step - loss: 1.1566 - accuracy: 0.4600 - val_loss: 1.0798 - val_accuracy: 0.5111
Epoch 59/100
27/27 [==============================] - 4s 164ms/step - loss: 1.1573 - accuracy: 0.4525 - val_loss: 1.0875 - val_accuracy: 0.5333
Epoch 60/100
27/27 [==============================] - 4s 165ms/step - loss: 1.1492 - accuracy: 0.4637 - val_loss: 1.0817 - val_accuracy: 0.5481
Epoch 61/100
27/27 [==============================] - 5s 169ms/step - loss: 1.1514 - accuracy: 0.4637 - val_loss: 1.0908 - val_accuracy: 0.5037
Epoch 62/100
27/27 [==============================] - 5s 172ms/step - loss: 1.1571 - accuracy: 0.4525 - val_loss: 1.0918 - val_accuracy: 0.5185
Epoch 63/100
27/27 [==============================] - 5s 171ms/step - loss: 1.1506 - accuracy: 0.4600 - val_loss: 1.0724 - val_accuracy: 0.5407
Epoch 64/100
27/27 [==============================] - 5s 172ms/step - loss: 1.1499 - accuracy: 0.4581 - val_loss: 1.0697 - val_accuracy:

In [42]:
loss, acc = model_LSTM.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 64ms/step - loss: 1.0023 - accuracy: 0.5185
Restored model, accuracy: 51.85%


In [48]:
#!rm -rf ./logs/
import tensorflow
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8960), started 1 day, 3:17:01 ago. (Use '!kill 8960' to kill it.)

In [49]:
!rm -rf ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.
